In [1]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

C:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    PARTICLES_DICT,
    TARGET_CODES,
    NUM_WORKERS,
)

In [4]:
config_common = {
    "bs": 512,
    "max_epochs": 40,
    "dropout": 0.1,
    "gamma": 0.9,
    "patience": 5,
    "patience_threshold": 0.001,
}

In [5]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cuda")

In [6]:
from pdi.data.preparation import FeatureSetPreparation, MeanImputation, DeletePreparation, RegressionImputation, EnsemblePreparation
from pdi.models import AttentionModel, NeuralNetEnsemble, NeuralNet
from pdi.data.constants import N_COLUMNS
from pdi.data.types import Split

EXPERIMENTS = {
    "Delete": {
        "data_preparation": DeletePreparation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "create_model": lambda d_prep: NeuralNet(
            [N_COLUMNS, wandb.config.h0, wandb.config.h1, wandb.config.h2, 1], nn.ReLU, wandb.config.dropout
        ).to(device),
    },
    "Mean": {
        "data_preparation": MeanImputation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "create_model": lambda d_prep: NeuralNet(
            [N_COLUMNS, wandb.config.h0, wandb.config.h1, wandb.config.h2, 1], nn.ReLU, wandb.config.dropout
        ).to(device),
    },
    "Regression": {
        "data_preparation": RegressionImputation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "create_model": lambda d_prep: NeuralNet(
            [N_COLUMNS, wandb.config.h0, wandb.config.h1, wandb.config.h2, 1], nn.ReLU, wandb.config.dropout
        ).to(device),
    },
    "Ensemble": {
        "data_preparation": EnsemblePreparation(),
         "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "create_model": lambda d_prep: NeuralNetEnsemble(
            d_prep.get_group_ids(),
            [wandb.config.h0, wandb.config.h1, wandb.config.h2, 1],
            nn.ReLU,
            wandb.config.dropout,
        ).to(device),
    },
    "Proposed": {
        "data_preparation": FeatureSetPreparation(),
        "config": {
            "embed_in": N_COLUMNS + 1,
            "embed_hidden": 128,
            "d_model": 32,
            "ff_hidden": 128,
            "pool_hidden": 64,
            "num_heads": 2,
            "num_blocks": 2, 
            "start_lr": 2e-4,
        },
        "create_model": lambda d_prep: AttentionModel(
            wandb.config.embed_in,
            wandb.config.embed_hidden,
            wandb.config.d_model,
            wandb.config.ff_hidden,
            wandb.config.pool_hidden,
            wandb.config.num_heads,
            wandb.config.num_blocks,
            nn.ReLU,
            wandb.config.dropout,
        ).to(device),
    },
}


In [7]:
import wandb
import os
from pdi.train import train
from pdi.constants import PARTICLES_DICT

def do_train(experiment_name, data_preparation, config, create_model):
    wandb_config = {**config_common, **config}

    train_loader, val_loader = data_preparation.prepare_dataloaders(wandb_config["bs"], NUM_WORKERS, [Split.TRAIN, Split.VAL])

    os.makedirs(f"models/{experiment_name}/", exist_ok=True)
    for target_code in TARGET_CODES:
        save_path = f"models/{experiment_name}/{PARTICLES_DICT[target_code]}"
        with wandb.init(project=experiment_name, config=wandb_config, name=PARTICLES_DICT[target_code]) as run:
            # pos_weight = torch.tensor(data_preparation.pos_weight(target_code)).float().to(device)
            pos_weight = torch.tensor(1.0).to(device)
            wandb.log({"pos_weight": pos_weight.item()})

            model = create_model(data_preparation)

            os.makedirs(f"models/{experiment_name}/", exist_ok=True)
            train(model, target_code, device, train_loader, val_loader, pos_weight)

            torch.save(model, f"{save_path}.pt")

In [ ]:
do_train("Delete", **EXPERIMENTS["Delete"])

In [ ]:
do_train("Mean", **EXPERIMENTS["Mean"])

In [ ]:
do_train("Regression", **EXPERIMENTS["Regression"])

In [ ]:
do_train("Ensemble", **EXPERIMENTS["Ensemble"])

In [8]:
do_train("FSE", **EXPERIMENTS["FSE"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mkasak. Use `wandb login --relogin` to force relogin


100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [00:13<00:00, 58.81it/s]


Epoch: 0, F1: 0.9773


100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [00:12<00:00, 63.36it/s]


Epoch: 1, F1: 0.9820


...

100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [00:12<00:00, 64.69it/s]


Epoch: 27, F1: 0.8426


100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [00:15<00:00, 52.92it/s]


Epoch: 28, F1: 0.8438
Finishing training early at epoch: 28


epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
loss,█▃▂▃▁▂▂▂▁▂▂▂▁▂▂▂▁▂▂▂▂▂▂▁▂▂▂▁▂▂▁▂▂▁▁▂▂▂▁▂
pos_weight,▁
scheduled_lr,█▇▇▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_f1,▁▄▆▆▇▇▇▅▇▇▇██████████████████
val_loss,█▆▅▃▃▂▂▄▂▃▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
val_precision,▂▃▂▁▆▅▃▁▆▄▅▅▇▅▃▄█▅▅▆▅▆▇█▄▇█▇▇
val_recall,▁▄▆█▅▇█▅▆▆▇▇▆▇██▆▇▇▇▇▇▇▆█▇▆▆▆
val_threshold,▇██▃█▄▂▄▄▆▃▅▄▃▁▂▅▂▃▄▄▄▄▅▂▄▅▅▅
epoch,28
loss,0.63284
